In [1]:
import env
from config import get_config
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [2]:
env_name = "MyHalfCheetah-v2"
env_str = "half_cheetah"
cfg = get_config(env_str)
env = cfg.exp_cfg.env

O = np.load('save/mpc_gym_true_dynamics_cmd_line/%s/obs.npy' % env_name)
A = np.load('save/mpc_gym_true_dynamics_cmd_line/%s/act.npy' % env_name)
print(O.shape, A.shape)
print(O.min(), O.max(), O.mean(), O.std())
print(A.min(), A.max(), A.mean(), A.std())
"""
State-Space (name/joint/parameter):
- rootx     slider      position (m)
- rootz     slider      position (m)
- rooty     hinge       angle (rad)
- bthigh    hinge       angle (rad)
- bshin     hinge       angle (rad)
- bfoot     hinge       angle (rad)
- fthigh    hinge       angle (rad)
- fshin     hinge       angle (rad)
- ffoot     hinge       angle (rad)
- rootx     slider      velocity (m/s)
- rootz     slider      velocity (m/s)
- rooty     hinge       angular velocity (rad/s)
- bthigh    hinge       angular velocity (rad/s)
- bshin     hinge       angular velocity (rad/s)
- bfoot     hinge       angular velocity (rad/s)
- fthigh    hinge       angular velocity (rad/s)
- fshin     hinge       angular velocity (rad/s)
- ffoot     hinge       angular velocity (rad/s)
"""
print(O[0, 200])

(10, 251, 18) (10, 250, 6)
-17.81324721891102 743.643107217635 27.302207677283963 93.83773237504153
-1.0 1.0 0.05811394256070287 0.7836239857847269
[448.578   0.736 324.273   0.585   0.52   -0.438  -0.278  -0.533  -0.235
  14.895   3.102   9.02   -4.575  -1.763  -0.872   5.8     8.199   2.9  ]


In [3]:
env_name = "MySwimmer-v2"
env_str = "swimmer"
cfg = get_config(env_str)
env = cfg.exp_cfg.env

O = np.load('save/mpc_gym_true_dynamics_cmd_line/%s/obs.npy' % env_name)
A = np.load('save/mpc_gym_true_dynamics_cmd_line/%s/act.npy' % env_name)
print(O.shape, A.shape)
print(O.min(), O.max(), O.mean(), O.std())
print(A.min(), A.max(), A.mean(), A.std())
print(O[0, 100])

(10, 112, 10) (10, 111, 2)
-5.977867925604655 9.726793612191898 0.4648059201985191 2.366158531725691
-1.0 1.0 -0.0022904544834106318 0.8002182222025644
[ 7.238 -4.088  0.118 -0.04  -1.544  0.441  1.809 -2.097  2.956  1.551]


In [4]:
env_name = "MyCartpole-v0"
env_str = "swimmer"
cfg = get_config(env_str)
env = cfg.exp_cfg.env

O = np.load('save/mpc_gym_true_dynamics_cmd_line/%s/obs.npy' % env_name)
A = np.load('save/mpc_gym_true_dynamics_cmd_line/%s/act.npy' % env_name)
print(O.shape, A.shape)
print(O.min(), O.max(), O.mean(), O.std())
print(A.min(), A.max(), A.mean(), A.std())
print(O[0, 30])

(10, 51, 4) (10, 50, 1)
-10.494429475837036 11.77005676253115 -0.16115197429940276 1.9265406681761197
-3.0 3.0 -0.0024171345789793662 1.0019499719419773
[ 0.005  3.161 -0.128  0.152]
